In [1]:
import numpy as np

In [2]:
fn       = np.loadtxt('MassiveNuS_params.txt',usecols=[0],dtype=np.str)
params   = np.loadtxt('MassiveNuS_params.txt',usecols=[1,2,3])## M_nu,Omega_m,A_s*1e9
data_sim = np.array([np.load('%s_Peaks_KN_s2.00_z1.00_ng40.00_b050.npy'%(ifn))[2:] for ifn in fn])

In [3]:
def paramsFromName(fn):
    Om=float(fn.split('Om')[1].split('_')[0])
    As=float(fn.split('As')[1].split('_')[0])
    mnv=float(fn.split('mva')[1].split('_')[0])
    mnv+=float(fn.split('mvb')[1].split('_')[0])
    mnv+=float(fn.split('mvc')[1].split('_')[0])
    return mnv,Om,As


PARAMS = [paramsFromName(f) for f in fn]

In [4]:
print(params.shape)
print(data_sim.shape)

(101, 3)
(101, 9999, 50)


In [5]:
params_fid = params[51]
theta_fiducial = params_fid

In [6]:
bin_centers = np.linspace(-1.92,5.92, 50)

In [7]:
np.diff(bin_centers)

array([0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
       0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
       0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
       0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
       0.16, 0.16, 0.16, 0.16, 0.16])

In [8]:
data_sim[0,0]*0.00126899073531

array([ 0.,  0.,  1.,  0.,  0.,  2.,  1.,  3.,  4.,  1., 10., 16., 12.,
       21., 20., 27., 38., 47., 49., 36., 26., 27., 20., 31., 24., 27.,
       22., 15.,  8., 18., 10.,  5.,  5.,  3.,  4.,  2.,  1.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [9]:
means = np.mean(data_sim,axis=1)

In [10]:
std  = np.std(data_sim[0,:,:],axis=0)

In [11]:
import sys

In [12]:
sys.path.append("/home/nessa/Documents/Projects/pydelfi/") 

In [13]:
import numpy as np
import ndes.ndes as ndes
import delfi.delfi as delfi
import compression.score.score as score
import distributions.priors as priors
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

In [14]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


kernel = C(1.0, (1e-4, 1e4)) * RBF(1, (1e-4, 1e4))
#Instanciate a Gaussian Process model
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
# Fit to data 

gp.fit(PARAMS,means)
#use the GP prediction to build the model
def fitGP(theta):
    pred, stdev = gp.predict(np.array(theta).reshape(1,3),return_std=True)
    return pred.T

h=0.01

theta1 = np.asarray([theta_fiducial[0]*(1+h), theta_fiducial[1],theta_fiducial[2]])
theta1_= np.asarray([theta_fiducial[0]*(1-h), theta_fiducial[1],theta_fiducial[2]])

theta2 = np.asarray([theta_fiducial[0], theta_fiducial[1]*(1+h),theta_fiducial[2]])
theta2_= np.asarray([theta_fiducial[0], theta_fiducial[1]*(1-h),theta_fiducial[2]])

theta3 = np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1+h)])
theta3_= np.asarray([theta_fiducial[0], theta_fiducial[1],theta_fiducial[2]*(1-h)])

dmudt1 = (fitGP(theta1)-fitGP(theta1_))/(theta1-theta1_)[0]
dmudt2 = (fitGP(theta2)-fitGP(theta2_))/(theta2-theta2_)[1]
dmudt3 = (fitGP(theta3)-fitGP(theta3_))/(theta3-theta3_)[2]

/home/nessa/miniconda2/envs/py3torch/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-4.47135305e+06, -2.61671294e+00]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 64, 'nit': 18, 'warnflag': 2}
  ConvergenceWarning)


In [15]:
dmudt = np.hstack((dmudt1,dmudt2,dmudt3)).T

In [16]:
theta_fiducial

array([0.1, 0.3, 2.1])

In [17]:
fid_index = 51

In [18]:
realizations_4_cov = np.load('Peaks_KN_s2.00_z1.00_ng40.00_b050.npy')

In [19]:
Cov     = np.cov(realizations_4_cov.T)
Cov_Inv = np.linalg.inv(Cov)

diag = np.diag(Cov)

diag1= np.expand_dims(diag,1)
diag2= np.expand_dims(diag,0)

In [20]:
# plt.imshow(Cov_Inv)
# plt.colorbar()
# plt.show()
# plt.imshow(Cov)
# plt.colorbar()
# plt.show()
# plt.imshow(Cov/np.sqrt(diag1*diag2))
# plt.colorbar()
# plt.show()

In [33]:
theta_fiducial = np.array(params_fid)
mu             = fitGP(theta_fiducial)[:,0]
Cinv           = Cov_Inv

Compressor     = score.Gaussian(len(mu), theta_fiducial, mu = mu, Cinv = Cinv, dmudt = dmudt)
Compressor.compute_fisher()
Finv           = Compressor.Finv

def compressor(d, compressor_args):
    return Compressor.scoreMLE(d)
compressor_args=None

In [30]:
ordered_params=[]
averaged_data =[]
averaged_train=[]
for ii in range(len(data_sim)):
    if ii==51:
        par = PARAMS[ii]
        print(par)
        for jj in range(np.int((data_sim.shape[1]+1)/100)):
            averaged_data+=[np.mean(data_sim[ii,jj*100:(jj+1)*100,:],axis=0)]
    else:
         for jj in range(np.int((data_sim.shape[1]+1)/100)):
            averaged_train+=[np.mean(data_sim[ii,jj*100:(jj+1)*100,:],axis=0)]
            ordered_params+=[PARAMS[ii]]
    
averaged_data = np.asarray(averaged_data)
averaged_train = np.asarray(averaged_train)
ordered_params=np.asarray(ordered_params)

(0.09999, 0.2978, 2.1)


In [31]:
for ii in range(len(PARAMS)):
    if PARAMS[ii][0]==0.0:
        print(ii)
        
PARAMS[51]

53


(0.09999, 0.2978, 2.1)

In [32]:
compressed_data = np.zeros((averaged_data.shape[0],3))

for ii in range(len(averaged_data)):
    compressed_data[ii] = compressor(averaged_data[ii],None)
    
compressed_train = np.zeros((averaged_train.shape[0],3))

for ii in range(len(averaged_train)):
    compressed_train[ii] = compressor(averaged_train[ii],None)

ValueError: setting an array element with a sequence.

In [25]:
print(compressed_data.shape,compressed_train.shape,ordered_params.shape)

(100, 3) (10000, 3) (10000, 3)


In [26]:
print(compressed_train[:,0].min(),compressed_train[:,0].max())
print(compressed_train[:,1].min(),compressed_train[:,1].max())
print(compressed_train[:,2].min(),compressed_train[:,2].max())

-0.059627439165817 0.8297655391749368
0.22564695135248536 0.405109429432674
1.6504816420936934 2.3019055567564735


In [27]:
lower = np.array([np.min(params[:,0]),np.min(params[:,1]),np.min(params[:,2])])
upper = np.array([np.max(params[:,0]),np.max(params[:,1]),np.max(params[:,2])])
# upper = np.array([1, 1.2, 0.1, 1.0, 1.3])
prior = priors.Uniform(lower, upper)

In [28]:
DelfiEnsemble = delfi.Delfi(compressed_data[0], prior, NDEs, 
                            Finv = Finv, 
                            theta_fiducial = theta_fiducial, 
                            param_limits = [lower, upper],
                            param_names = ['M_\nu', '\Omega_m', 'A_s'], 
                            results_dir = "./",
                            input_normalization="fisher")

In [29]:
DelfiEnsemble.load_simulations(compressed_train,ordered_params)

In [ ]:
DelfiEnsemble.fisher_pretraining()

In [ ]:
DelfiEnsemble.train_ndes()

In [ ]:
posterior_samples = DelfiEnsemble.emcee_sample()